<a href="https://colab.research.google.com/github/86HenriqueSilva/Treinamento_de_ia-s/blob/main/IA_PyTorch_REDE_NEURAL_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# Montar o Google Drive para acessar o arquivo de dados
from google.colab import drive
drive.mount('/content/drive')

# URL do arquivo CSV no Google Drive contendo os dados do histórico da Mega Sena
url = "https://drive.google.com/uc?id=1BXzIxSNSK03Q73OfvC_CCDoJSOZ_LI2n"

# Carregar os dados do arquivo CSV para uma estrutura de dados tabular
print("Carregando os dados do histórico da Mega Sena...")
dados = pd.read_csv(url)

# Remover colunas de data
dados = dados.drop(columns=['Data'])

# Definir janela de histórico (quantidade de concursos anteriores)
janela = 10

# Função para criar janelas de histórico dos dados
def criar_janelas(dados, janela):
    """
    Função para criar janelas de histórico dos dados.

    Args:
        dados (DataFrame): DataFrame contendo os dados do histórico da Mega Sena.
        janela (int): Tamanho da janela de histórico.

    Returns:
        X (array): Array contendo as janelas de histórico.
        y (array): Array contendo os dados do próximo concurso após cada janela.
    """
    X, y = [], []
    for i in range(len(dados)-janela-1):
        janela_dados = dados.iloc[i:(i+janela), 1:].values.flatten()
        X.append(janela_dados)
        proximo_concurso = dados.iloc[i+janela, 1:].values
        y.append(proximo_concurso)
    return np.array(X), np.array(y)

# Criar janelas de histórico dos dados
X, y = criar_janelas(dados, janela)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Definir modelo da rede neural
class MegaSenaPredictor(nn.Module):
    """
    Classe para o modelo da rede neural para previsão da Mega Sena.

    Args:
        input_size (int): Tamanho da entrada.
        hidden_size (int): Tamanho da camada oculta.
        output_size (int): Tamanho da saída.
    """
    def __init__(self, input_size, hidden_size, output_size):
        super(MegaSenaPredictor, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Definir hiperparâmetros da rede neural
input_size = janela * 6  # 6 bolas em cada concurso
hidden_size = 1024
output_size = 6

# Instanciar o modelo da rede neural
modelo = MegaSenaPredictor(input_size, hidden_size, output_size)

# Definir função de perda e otimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(modelo.parameters(), lr=0.001)

# Definir número de épocas e tamanho do batch
num_epochs = 1000
batch_size = 512

# Treinar a rede neural
print("Treinando a rede neural...")
for epoch in tqdm(range(num_epochs)):
    # Embaralhar os dados de treinamento
    indices = np.random.permutation(len(X_train))
    X_train_shuffled = X_train[indices]
    y_train_shuffled = y_train[indices]

    # Dividir os dados em batches
    for i in range(0, len(X_train), batch_size):
        inputs = torch.tensor(X_train_shuffled[i:i+batch_size], dtype=torch.float32)
        targets = torch.tensor(y_train_shuffled[i:i+batch_size], dtype=torch.float32)

        # Forward pass
        optimizer.zero_grad()
        outputs = modelo(inputs)
        loss = criterion(outputs, targets)

        # Backward pass e otimização
        loss.backward()
        optimizer.step()

print("Treinamento concluído.")

# Avaliar o modelo
modelo.eval()
with torch.no_grad():
    inputs = torch.tensor(X_test, dtype=torch.float32)
    outputs = modelo(inputs).numpy()

# Exibir previsões
print("Previsões para o próximo sorteio da Mega Sena:")
previsoes_inteiras = outputs[0].round().astype(int)
print(previsoes_inteiras)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Carregando os dados do histórico da Mega Sena...
Treinando a rede neural...


100%|██████████| 1000/1000 [00:30<00:00, 33.28it/s]

Treinamento concluído.
Previsões para o próximo sorteio da Mega Sena:
[47 11 24 13 42 56]
